## Code to extract historical Amazon data from Keepa using an API 

In [ ]:
pip install keepa

In [2]:
#Convert keepa time to unix time
def keepaTimeMinutesToUnixTime(keepaMinutes):
    return (21564000 + keepaMinutes) * 60000;

In [11]:
#import necessary libraries
from datetime import datetime, timedelta
import keepa, json
import pandas as pd

#We need API Key from Keepa
KEEPA_API_KEY = "atbrg0mqdcl26i3dt5453hng6h82e66i9j730vc7d1bs1timtshn448ns6uslh5l"

#Define api object
api = keepa.Keepa(KEEPA_API_KEY)

#Get how many tokens we have
print('Tokens Left:', api.tokens_left)
# If you'll receive an answer like this
# Tokens Left: 300
# ... key is correct and API is working

#Define our asins list
asins_list = ['B003UKM9CO']

#Call product info with prices from Keepa
products = api.query(
      asins_list,
      progress_bar = False
)


#Define result frame with all data
df_result = pd.DataFrame()

#Iterate through all products in the result
for product in products:

     # Just show info about product
     print(f"{product['asin']} | {product['title']}")

     # Get prices info
     csv = product['csv']
     prices_data = csv[1]

     #Transform the prices data into a list
     transformed_prices = [(datetime.utcfromtimestamp(keepaTimeMinutesToUnixTime(keepaMinutes) / 1000), val/100 if val>100 else val) for
                           keepaMinutes, val in zip(prices_data[::2], prices_data[1::2])]

     # Create dataframe with prices from transformed_prices
     df_prices = pd.DataFrame(transformed_prices, columns=['date', 'price'])
     df_prices.set_index('date', inplace=True)

     # Resample dataframe by days and forward fill to handle NaN values
     df_prices = df_prices.resample('D').apply(lambda x: x[x > -1].mean()).ffill()

     # Extend the DataFrame to include all days up to the current day
     if not df_prices.empty:
          last_date = df_prices.last_valid_index()
          current_date = pd.to_datetime("today").normalize()  # Normalize the time to 00:00:00
          new_index = pd.date_range(start=df_prices.index.min(), end=current_date, freq='D')
          df_prices = df_prices.reindex(new_index, method='ffill')  # Reindex and fill all missing days

     #Add columns about asin and market
     df_prices['asin'] = product['asin']
     df_prices['market'] = 'us' #need, if we want to have multiple market products in one dataframe
     df_prices = df_prices.rename_axis('date').reset_index() #we need to move date from index

     #Concat df_prices to one frame
     df_result = pd.concat([df_result, df_prices], ignore_index=True)

#Show result frame
print(df_result)

#Save result frame to csv
file_path = '/Users/sofialozano/Library/CloudStorage/OneDrive-EmoryUniversity/EMORY FALL 2024/ECON 521 POLICY CAUSAL INFERENCE/Final Project 521/521 Prices Amazon keepa/NEW_toothbrush_amazon_asins.csv'
df_result.to_csv(file_path)

Tokens Left: 59
B003UKM9CO | Oral-B Pro 1000 Rechargeable Electric Toothbrush, White
           date  price        asin market
0    2015-07-26  34.00  B003UKM9CO     us
1    2015-07-27  34.00  B003UKM9CO     us
2    2015-07-28  34.00  B003UKM9CO     us
3    2015-07-29  34.00  B003UKM9CO     us
4    2015-07-30  34.00  B003UKM9CO     us
...         ...    ...         ...    ...
3415 2024-11-30  39.94  B003UKM9CO     us
3416 2024-12-01  38.76  B003UKM9CO     us
3417 2024-12-02  39.94  B003UKM9CO     us
3418 2024-12-03  39.94  B003UKM9CO     us
3419 2024-12-04  39.80  B003UKM9CO     us

[3420 rows x 4 columns]
